# 手撕常见算法面试题目
##### zhiyue_mouse最爱版


### 线性回归

#### 原理
线性回归假设目标变量y与特征x之间存在线性关系：


$y = Xw + b$，其中$X$是特征矩阵，$w$是权重向量，$b$是偏置。


损失函数为MSE：


$J(w,b) = \frac{1}{2m}\sum_{i=1}^{m}(h_w(x^{(i)}) - y^{(i)})^2$

梯度下降更新：
$w_j = w_j - \alpha\frac{\partial J}{\partial w_j} = w_j - \alpha\frac{1}{m}\sum_{i=1}^{m}(h_w(x^{(i)}) - y^{(i)})x_j^{(i)}$

$b = b - \alpha\frac{\partial J}{\partial b} = b - \alpha\frac{1}{m}\sum_{i=1}^{m}(h_w(x^{(i)}) - y^{(i)})$

In [10]:
import numpy as np

class LinearRegression:
    def __init__(self, learning_rate=0.01, n_iters=1000):
        self.lr = learning_rate
        self.n_iters = n_iters
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0
        for _ in range(self.n_iters):
            y_pred = np.dot(X, self.weights) + self.bias
            dw = (1 / n_samples) * np.dot(X.T, (y_pred - y))
            db = (1 / n_samples) * np.sum(y_pred - y)
            self.weights -= self.lr * dw
            self.bias -= self.lr * db
            if _ % 100 == 0:
                print(f"Iteration {_+1}: Weights: {self.weights}, Bias: {self.bias}")
    def predict(self, X):
        return np.dot(X, self.weights) + self.bias
    
    def mean_squared_erorr(self, y_true, y_pred):
        return np.mean((y_true - y_pred) ** 2)
    
X = np.array([[1, 1], [2, 2], [3, 3], [4, 4]])
y = np.array([2, 4, 6, 8])
regressor = LinearRegression(learning_rate=0.01, n_iters=1000)
regressor.fit(X, y)
predictions = regressor.predict(X)
print("Predictions:", predictions)
print("Mean Squared Error:", regressor.mean_squared_erorr(y, predictions))

Iteration 1: Weights: [0.15 0.15], Bias: 0.05
Iteration 101: Weights: [0.9542206 0.9542206], Bias: 0.2717865473927469
Iteration 201: Weights: [0.96090859 0.96090859], Bias: 0.23208088730299303
Iteration 301: Weights: [0.96661951 0.96661951], Bias: 0.1981758738605035
Iteration 401: Weights: [0.97149611 0.97149611], Bias: 0.16922409008674807
Iteration 501: Weights: [0.97566028 0.97566028], Bias: 0.14450191190197753
Iteration 601: Weights: [0.97921611 0.97921611], Bias: 0.12339143045545621
Iteration 701: Weights: [0.98225245 0.98225245], Bias: 0.10536500804343571
Iteration 801: Weights: [0.98484522 0.98484522], Bias: 0.08997209027413752
Iteration 901: Weights: [0.9870592 0.9870592], Bias: 0.07682794486154712
Predictions: [2.04357228 4.02143683 5.99930138 7.97716592]
Mean Squared Error: 0.0007199911707355395


#### 逻辑回归
原理

Sigmoid函数：

$\sigma(z) = \frac{1}{1 + e^{-z}}$

假设函数（交叉熵）：

$J(w,b) = -\frac{1}{m} \sum_{i=1}^{m}[y^{(i)}\log(h_w(x^{(i)})) + (1-y^{(i)}\log(1-h_w(x^{(i)})))]$

梯度更新：

$w_j = w_j - \alpha\frac{\partial J}{\partial w_j} = w_j - \alpha\frac{1}{m}\sum_{i=1}^{m}{h_w(x^{(i)})x_j^{(i)}}$


In [11]:
class LogisticRegression:
    def __init__(self, learning_rate=0.01, n_iters=1000):
        self.lr = learning_rate
        self.n_iters = n_iters
        self.weights = None
        self.bias = None
        
    def _sigmoid(self, z):
        return 1 / (1 + np.exp(-z))
    
    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0
        
        # 梯度下降
        for i in range(self.n_iters):
            linear_model = np.dot(X, self.weights) + self.bias
            y_pred = self._sigmoid(linear_model)
            
            # 计算梯度
            dw = (1/n_samples) * np.dot(X.T, (y_pred - y))
            db = (1/n_samples) * np.sum(y_pred - y)
            
            # 更新参数
            self.weights -= self.lr * dw
            self.bias -= self.lr * db
            
            if i % 100 == 0:
                # 计算损失
                loss = -np.mean(y * np.log(y_pred) + (1-y) * np.log(1-y_pred))
                print(f"Iteration {i}, Loss: {loss:.4f}")
    
    def predict_proba(self, X):
        linear_model = np.dot(X, self.weights) + self.bias
        return self._sigmoid(linear_model)
    
    def predict(self, X, threshold=0.5):
        return self.predict_proba(X) >= threshold

# 测试逻辑回归
print("\n=== Logistic Regression ===")
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

X, y = make_classification(n_samples=100, n_features=2, n_redundant=0, 
                          n_informative=2, random_state=42)

# 分割数据集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 训练
log_reg = LogisticRegression(learning_rate=0.1, n_iters=1000)
log_reg.fit(X_train, y_train)

# 预测
y_pred = log_reg.predict(X_test)
accuracy = np.mean(y_pred == y_test)
print(f"Test Accuracy: {accuracy:.4f}")

# 输出概率
y_proba = log_reg.predict_proba(X_test)
print("Sample predictions (probability, prediction, true):")
for i in range(5):
    print(f"  {y_proba[i]:.4f}, {y_pred[i]}, {y_test[i]}")


=== Logistic Regression ===
Iteration 0, Loss: 0.6931
Iteration 100, Loss: 0.1720
Iteration 200, Loss: 0.1215
Iteration 300, Loss: 0.1006
Iteration 400, Loss: 0.0888
Iteration 500, Loss: 0.0810
Iteration 600, Loss: 0.0753
Iteration 700, Loss: 0.0710
Iteration 800, Loss: 0.0676
Iteration 900, Loss: 0.0649
Test Accuracy: 0.9500
Sample predictions (probability, prediction, true):
  0.0021, False, 0
  0.9802, True, 1
  0.9969, True, 1
  0.0151, False, 0
  0.9983, True, 1


#### Multi-head Self-Attention
原理公式
查询、键、值投影：

$Q = XW^Q$, $K = XW^K$, $V = XW^V$

缩放点积注意力：

$\text{Attention}(Q,K,V) = \text{softmax}(\frac{QK^T}{\sqrt{d_k}})V$

多头注意力：

$\text{MultiHead}(Q,K,V) = \text{Concat}(head_1,...,head_h)W^O$

其中 

$head_i = \text{Attention}(QW_i^Q, KW_i^K, VW_i^V)$

In [18]:
import math
import numpy as np

class MultiHeadSelfAttention:
    def __init__(self, d_model, num_heads):
        self.d_model = d_model
        self.num_heads = num_heads
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        self.d_k = d_model // num_heads
        
        # 权重矩阵 - 修正维度
        self.W_q = np.random.randn(d_model, d_model) * 0.01
        self.W_k = np.random.randn(d_model, d_model) * 0.01
        self.W_v = np.random.randn(d_model, d_model) * 0.01
        self.W_o = np.random.randn(d_model, d_model) * 0.01
        
    def softmax(self, x):
        exp_x = np.exp(x - np.max(x, axis=-1, keepdims=True))
        return exp_x / np.sum(exp_x, axis=-1, keepdims=True)
    
    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        # Q, K, V 形状: (batch_size, num_heads, seq_len, d_k)
        
        # 修正矩阵乘法维度
        # K 需要转置最后两个维度: (batch_size, num_heads, d_k, seq_len)
        scores = np.matmul(Q, K.swapaxes(-2, -1)) / math.sqrt(self.d_k)
        
        if mask is not None:
            scores = scores + mask * -1e9
            
        attention_weights = self.softmax(scores)
        output = np.matmul(attention_weights, V)  # (batch_size, num_heads, seq_len, d_k)
        return output, attention_weights
    
    def forward(self, x, mask=None):
        batch_size, seq_len, d_model = x.shape
        
        # 线性投影
        Q = np.dot(x, self.W_q)  # (batch_size, seq_len, d_model)
        K = np.dot(x, self.W_k)
        V = np.dot(x, self.W_v)
        
        # 重塑为多头 - 修正维度变换
        Q = Q.reshape(batch_size, seq_len, self.num_heads, self.d_k).transpose(0, 2, 1, 3)
        K = K.reshape(batch_size, seq_len, self.num_heads, self.d_k).transpose(0, 2, 1, 3)
        V = V.reshape(batch_size, seq_len, self.num_heads, self.d_k).transpose(0, 2, 1, 3)
        # 现在形状: (batch_size, num_heads, seq_len, d_k)
        
        # 计算注意力
        attention_output, attention_weights = self.scaled_dot_product_attention(Q, K, V, mask)
        
        # 合并多头 - 修正维度变换
        attention_output = attention_output.transpose(0, 2, 1, 3)  # (batch_size, seq_len, num_heads, d_k)
        attention_output = attention_output.reshape(batch_size, seq_len, d_model)
        
        # 输出投影
        output = np.dot(attention_output, self.W_o)
        
        return output, attention_weights

# 测试修复后的Multi-head Self-Attention
print("\n=== Fixed Multi-head Self-Attention ===")
# 创建示例数据
batch_size, seq_len, d_model = 2, 4, 8
num_heads = 2

x = np.random.randn(batch_size, seq_len, d_model)
print(f"Input shape: {x.shape}")

# 初始化注意力层
attention = MultiHeadSelfAttention(d_model, num_heads)
output, attn_weights = attention.forward(x)

print(f"Output shape: {output.shape}")
print(f"Attention weights shape: {attn_weights.shape}")

# 显示注意力权重
print("Attention weights for first head, first batch:")
print(attn_weights[0, 0].round(3))

# 验证输出
print(f"\nInput range: [{x.min():.3f}, {x.max():.3f}]")
print(f"Output range: [{output.min():.3f}, {output.max():.3f}]")


=== Fixed Multi-head Self-Attention ===
Input shape: (2, 4, 8)
Output shape: (2, 4, 8)
Attention weights shape: (2, 2, 4, 4)
Attention weights for first head, first batch:
[[0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]]

Input range: [-2.025, 3.853]
Output range: [-0.001, 0.001]


#### K-means 聚类
原理公式
目标函数：

$J = \sum_{i=1}^{k}\sum_{x\in C_i}||x - \mu_i||^2$

质心更新：

$\mu_i = \frac{1}{|C_i|}\sum_{x\in C_i}x$

In [14]:
class KMeans:
    def __init__(self, k=3, max_iters=100):
        self.k = k
        self.max_iters = max_iters
        self.centroids = None
        self.labels = None
        
    def _initialize_centroids(self, X):
        n_samples = X.shape[0]
        random_indices = np.random.choice(n_samples, self.k, replace=False)
        return X[random_indices]
    
    def _assign_clusters(self, X):
        distances = np.zeros((X.shape[0], self.k))
        for i in range(self.k):
            distances[:, i] = np.linalg.norm(X - self.centroids[i], axis=1)
        return np.argmin(distances, axis=1)
    
    def _update_centroids(self, X, labels):
        new_centroids = np.zeros((self.k, X.shape[1]))
        for i in range(self.k):
            if np.sum(labels == i) > 0:
                new_centroids[i] = X[labels == i].mean(axis=0)
            else:
                new_centroids[i] = self.centroids[i]  # 保持原质心
        return new_centroids
    
    def fit(self, X):
        self.centroids = self._initialize_centroids(X)
        
        for i in range(self.max_iters):
            # 分配簇
            labels = self._assign_clusters(X)
            
            # 更新质心
            new_centroids = self._update_centroids(X, labels)
            
            # 检查收敛
            if np.allclose(self.centroids, new_centroids):
                print(f"Converged at iteration {i}")
                break
                
            self.centroids = new_centroids
            
            if i % 10 == 0:
                inertia = self._compute_inertia(X, labels)
                print(f"Iteration {i}, Inertia: {inertia:.4f}")
        
        self.labels = self._assign_clusters(X)
        return self
    
    def _compute_inertia(self, X, labels):
        inertia = 0
        for i in range(self.k):
            cluster_points = X[labels == i]
            if len(cluster_points) > 0:
                inertia += np.sum((cluster_points - self.centroids[i])**2)
        return inertia
    
    def predict(self, X):
        return self._assign_clusters(X)

# 测试K-means
print("\n=== K-means Clustering ===")
from sklearn.datasets import make_blobs

# 生成测试数据
X, y_true = make_blobs(n_samples=300, centers=3, n_features=2, 
                       random_state=42, cluster_std=1.0)

# 训练K-means
kmeans = KMeans(k=3, max_iters=100)
kmeans.fit(X)

# 预测
y_pred = kmeans.labels

print("Centroids:")
for i, centroid in enumerate(kmeans.centroids):
    print(f"  Cluster {i}: {centroid}")

# 计算准确率（需要调整标签映射）
from sklearn.metrics import adjusted_rand_score
ari = adjusted_rand_score(y_true, y_pred)
print(f"Adjusted Rand Index: {ari:.4f}")

# 显示每个簇的大小
unique, counts = np.unique(y_pred, return_counts=True)
print("Cluster sizes:", dict(zip(unique, counts)))


=== K-means Clustering ===
Iteration 0, Inertia: 8998.7533
Converged at iteration 2
Centroids:
  Cluster 0: [-6.88387179 -6.98398415]
  Cluster 1: [4.74710337 2.01059427]
  Cluster 2: [-2.63323268  9.04356978]
Adjusted Rand Index: 1.0000
Cluster sizes: {np.int64(0): np.int64(100), np.int64(1): np.int64(100), np.int64(2): np.int64(100)}


#### AUC 计算
原理公式
$AUC = \frac{\sum_{i=1}^{m}\sum_{j=1}^{n}I(p_i > p_j)}{m \times n}$

其中 $I$ 是指示函数，$m$ 是正样本数，$n$ 是负样本数

In [16]:
def calculate_auc(y_true, y_scores):
    """手撕AUC计算"""
    # 将正负样本分开
    pos_scores = y_scores[y_true == 1]
    neg_scores = y_scores[y_true == 0]
    
    # 计算比较对
    count = 0
    for pos_score in pos_scores:
        for neg_score in neg_scores:
            if pos_score > neg_score:
                count += 1
            elif pos_score == neg_score:
                count += 0.5
    
    auc = count / (len(pos_scores) * len(neg_scores))
    return auc

def roc_curve(y_true, y_scores):
    """计算ROC曲线点"""
    # 按分数排序
    sorted_indices = np.argsort(y_scores)[::-1]
    y_true_sorted = y_true[sorted_indices]
    y_scores_sorted = y_scores[sorted_indices]
    
    # 计算TPR和FPR
    tpr, fpr = [0], [0]
    tp, fp = 0, 0
    total_p = np.sum(y_true == 1)
    total_n = np.sum(y_true == 0)
    
    for i in range(len(y_true_sorted)):
        if y_true_sorted[i] == 1:
            tp += 1
        else:
            fp += 1
        
        tpr.append(tp / total_p)
        fpr.append(fp / total_n)
    
    return fpr, tpr

# 测试AUC计算
print("\n=== AUC Calculation ===")
# 生成示例数据
np.random.seed(42)
n_samples = 100
y_true = np.random.randint(0, 2, n_samples)
y_scores = np.random.rand(n_samples)

# 计算AUC
auc_manual = calculate_auc(y_true, y_scores)
print(f"Manual AUC: {auc_manual:.4f}")

# 使用sklearn验证
from sklearn.metrics import roc_auc_score, roc_curve as sk_roc_curve
auc_sklearn = roc_auc_score(y_true, y_scores)
print(f"Sklearn AUC: {auc_sklearn:.4f}")

# 计算ROC曲线
fpr, tpr = roc_curve(y_true, y_scores)
print(f"ROC curve points: {len(fpr)}")

# 显示前几个点
print("First 5 ROC points (FPR, TPR):")
for i in range(min(5, len(fpr))):
    print(f"  ({fpr[i]:.3f}, {tpr[i]:.3f})")


=== AUC Calculation ===
Manual AUC: 0.5122
Sklearn AUC: 0.5122
ROC curve points: 101
First 5 ROC points (FPR, TPR):
  (0.000, 0.000)
  (0.023, 0.000)
  (0.023, 0.018)
  (0.045, 0.018)
  (0.045, 0.036)
